In [1]:
# keras module for building LSTM
import json
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

import tensorflow as tf

import numpy as np

import warnings
import matplotlib.pyplot as plt
from keras.preprocessing.text import tokenizer_from_json
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

from dataset import load_tokenized_sentences

LEN_MIN_LIMIT = 5
LEN_MAX_LIMIT = 45
SKIP = 1

dataset = load_tokenized_sentences('../datasets/words/bajki-raw.pickle')

with open('../datasets/words/books-bajki-raw-tokenizer_100000.json', 'r') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
        
print({
    "size": len(dataset),
    "tokenizer": tokenizer.num_words,
    "str": dataset[1][:500],
    "tokenizer": tokenizer.num_words
})

{'size': 39, 'tokenizer': 100000, 'str': 'Jacob i Wilhelm Grimm\n\nCzerwony Kapturek\ntłum. Marceli Tarnowski\n\nISBN 978-83-288-2269-6\n\n\nByła raz mała słodka dzieweczka, którą kochał każdy, kto ją tylko ujrzał, a najwięcej kochała ją babcia — nie wiedziała wprost, co jej dać. Pewnego razu podarowała jej kapturek z czerwonego aksamitu, a dziewczynce tak się ten kapturek podobał, że nie chciała nosić żadnego innego, toteż nazwano ją Czerwonym Kapturkiem.\n\nPewnego razu rzekła matka do Czerwonego Kapturka:\n\n— Oto masz, dziecko, w koszyku placek'}


In [2]:
from typing import List


def get_sequence_of_tokens(dataset: List[str], tokenizer: Tokenizer):
    word2int_sequences = []
    for seq_text in tokenizer.texts_to_sequences(dataset):
        for sindex in range(0, len(seq_text) - LEN_MAX_LIMIT, SKIP):
            rlen = np.random.randint(LEN_MIN_LIMIT, LEN_MAX_LIMIT)
            n_gram_sequence = seq_text[sindex:sindex + rlen]
            word2int_sequences.append(n_gram_sequence)
    return word2int_sequences


def generate_padded_sequences(word2int_sequences: List[List[int]]):
    word2int_sequences = np.array(
        pad_sequences(word2int_sequences, maxlen=LEN_MAX_LIMIT, padding="pre")
    )
    predictors, label = word2int_sequences[:, :-1], word2int_sequences[:, -1]
    return predictors, label


def dataset_generator(dataset: List[str], tokenizer: Tokenizer, batch_size=256):
    flag = True
    while True:
        word2int_sequences = get_sequence_of_tokens(dataset, tokenizer)
        predictors, label = generate_padded_sequences(word2int_sequences)

        if flag:
            flag = False
            print('word2int_sequences', len(word2int_sequences))
        
        p = np.random.permutation(len(predictors))
        for i in range(0, len(predictors) - batch_size + 1, batch_size):
            indexes = p[i : i + batch_size]
            yield predictors[indexes], label[indexes]

total_words = tokenizer.num_words + 1
batch_size = 256
gen = dataset_generator(dataset, tokenizer, 256)
a, b = next(gen)
print(a.shape, b.shape)

word2int_sequences 493248
(256, 44) (256,)


In [11]:
class PredictCallback(tf.keras.callbacks.Callback):
    def __init__(self, seed_text: str, next_words: int, max_sequence_len: int, temperature = 0.0, model = None):
        self.seed_text = seed_text
        self.next_words = next_words
        self.max_sequence_len = max_sequence_len
        self.temperature = temperature
        if model is not None:
            self.model = model

    def sample(self, preds: np.ndarray):
        if self.temperature > 0:
            preds = np.asarray(preds).astype("float64")
            preds = np.log(preds) / self.temperature
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            preds = np.random.multinomial(1, preds, 1)
        return np.argmax(preds)

    def generate_text(self, _seed_text: str = None):
        seed_text: str = self.seed_text if _seed_text is None else _seed_text
        for _ in range(self.next_words):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences(
                [token_list], maxlen=self.max_sequence_len - 1, padding="pre"
            )
            predicted = self.model.predict_on_batch(token_list)[0]
            predicted = self.sample(predicted)
            try:
                output_word = tokenizer.index_word[predicted]
            except:
                output_word = ""
            seed_text += " " + output_word
        return seed_text
    
    def on_epoch_begin(self, epoch):
        seed_text = self.generate_text()
        print(f"Start epoch {epoch} of training; Temperature: {self.temperature:.1f} Generated text:", seed_text)

In [3]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(total_words, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
    return model

print(total_words, LEN_MAX_LIMIT)
model = create_model(LEN_MAX_LIMIT, total_words)
model.load_weights('../lstm_models/model_best_3.h5')

for layer in model.layers[:4]:
    layer.trainable = False
model.summary()
"""
1. dodanie do zbioru znaków .,?!;:
2. dodanie temperatury
3. usunięcie niektórych słów 
4. dodanie bidirectional
5. ograniczenie wyjścia dense 
6. przygotowanie zbioru
"""


100001 45
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 44, 50)            5000050   
                                                                 
 bidirectional (Bidirectiona  (None, 44, 512)          628736    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 44, 1024)         4198400   
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 44, 1024)          0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             6295552   
 nal)                                                            
                                              

'\n1. dodanie do zbioru znaków .,?!;:\n2. dodanie temperatury\n3. usunięcie niektórych słów \n4. dodanie bidirectional\n5. ograniczenie wyjścia dense \n6. przygotowanie zbioru\n'

In [4]:
model.fit(
    gen,
    steps_per_epoch=493638 // batch_size,
    epochs=3,
    verbose=1,
    callbacks=[
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0.1),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0.2),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0.5),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 1.0),
        tf.keras.callbacks.ModelCheckpoint('../lstm_models/model_best_3_retrain.h5', monitor='loss', save_best_only=True, save_weights_only=False)
    ],
)

Start epoch 0 of training; Temperature: 0.0 Generated text: dawno temu czerwony kapturek poszedł do lasu i popędził do domu. kiedy weszła do środka, ujrzała w ciemnościach worka i rzekła:

— chodź do domu i wypij za ramiona.

— a czego chcesz ode mnie, to nie mogę — odparł kopciuch — ale jeśli
Start epoch 0 of training; Temperature: 0.1 Generated text: dawno temu czerwony kapturek poszedł do lasu i popędził do domu. kiedy weszła do środka, ujrzała w ciemnościach worka i rzekła:

— nie mogę ci powiedzieć, że dziadek do orzechów jest młodym siostrzeńcem i siostrzeńcem ojca chrzestnego, i dziadka do orzechów, który miał
Start epoch 0 of training; Temperature: 0.2 Generated text: dawno temu czerwony kapturek poszedł do lasu i popędził do domu. kiedy doszedł do dużego zamku. spotkał go z dala, a potem położył się na trawie i zaczął się na dziedzińcu obiema rękami, a potem donośnym głosem rzekł:

— ojcze chrzestny, oto jest
Start epoch 0 of training; Temperature: 0.5 Generated text: dawno te

In [13]:
from tqdm import tqdm
text_seed = "dawno temu czerwony kapturek poszedł do lasu"
for temperature in tqdm([0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0]):
    pr = PredictCallback(text_seed, 250, LEN_MAX_LIMIT, temperature, model)
    text = pr.generate_text()
    with open(f'../generated_texts/text_temperature_{temperature:.2f}', 'w') as f:
        f.write(f'Seed: {text_seed}\n')
        f.write(text)
    


100%|██████████| 7/7 [00:36<00:00,  5.15s/it]
